In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import joblib

In [2]:
# Load the model from the file
loaded_data = joblib.load(f"train_test_data.pkl")

X_train = loaded_data['X_train']
X_test = loaded_data['X_test']
y_train = loaded_data['y_train']
y_test = loaded_data['y_test']
model = loaded_data['model']
mean_icr_by_part = loaded_data['mean_icr_by_part']
scaler = loaded_data['scaler']

In [3]:

# 5. Classyfication
def classify_dose_and_suggest(glucose_post, insulin, ww, glucose_pre, mean_icr_by_part, part_of_day, sensitivity=40, target_glucose=110):
    insulin_per_ww = insulin / ww if ww > 0 else 0
    mean_icr = mean_icr_by_part.get(part_of_day, 1.0)

    if insulin_per_ww > 2 * mean_icr:
        feedback = 2
    elif insulin_per_ww < 0.5 * mean_icr and (glucose_post > 150 or (glucose_pre > 120 and glucose_post > 130)):
        feedback = -1
    elif insulin_per_ww < 0.5 * mean_icr and glucose_post > 180:
        feedback = -2
    elif glucose_post < 70:
        feedback = 2
    elif 70 <= glucose_post < 80:
        feedback = 1
    elif 80 <= glucose_post <= 150:
        feedback = 0
    elif 150 < glucose_post <= 180:
        feedback = -1
    elif 180 < glucose_post <= 240:
        feedback = -2
    else:
        feedback = -3

    # Estimated dose
    glucose_diff = glucose_post - target_glucose
    correction = glucose_diff / sensitivity  # Positive if too high, negative if too low
    carb_insulin = ww * mean_icr  # Base dose on WW
    pre_correction = max(0, (glucose_pre - 120) / sensitivity)  # Correction for high glucose before meal
    suggested_insulin = max(0, carb_insulin + pre_correction + correction)  # Suggested dose

    return feedback, suggested_insulin

In [4]:
test_cases = [
    [150, 2, 5, 10, 5, 2/5 if 5 > 0 else 0, 9.5, 1],
    [100, 5, 3, 5, 2, 5/3 if 3 > 0 else 0, 12.0, 0],
    [200, 15, 6, 20, 10, 15/6 if 6 > 0 else 0, 18.5, 2],
    [90, 2, 2, 5, 2, 2/2 if 2 > 0 else 0, 22.0, 3],
    [180, 25, 5, 15, 8, 25/5 if 5 > 0 else 0, 8.0, 1],
    [140, 6, 4, 10, 5, 6/4 if 4 > 0 else 0, 14.0, 2],
]

test_df = pd.DataFrame(test_cases, columns=['glucose_pre', 'insulin', 'WW', 'fats', 'prot', 'insulin_per_ww', 'hour', 'part_of_day'])
test_scaled = scaler.transform(test_df)
glucose_predictions = model.predict(test_scaled)

feedback_desc = {
    -3: "Significantly too low dose",
    -2: "Too low dose",
    -1: "Slightly too low dose",
    0: "OK",
    1: "Slightly too high dose",
    2: "Too high dose"
}

part_of_day_desc = {
    0: "Morning (6:00-11:59)",
    1: "Afternoon (12:00-17:59)",
    2: "Evening (18:00-23:59)",
    3: "Night (0:00-5:59)"
}

for i, (glucose_pred, test_case) in enumerate(zip(glucose_predictions, test_cases)):
    glucose_pre = test_case[0]
    insulin = test_case[1]
    ww = test_case[2]
    fats = test_case[3]
    prot = test_case[4]
    hour = test_case[6]
    part_of_day = test_case[7]
    feedback, suggested_insulin = classify_dose_and_suggest(glucose_pred, insulin, ww, glucose_pre, mean_icr_by_part, part_of_day)

    print(f"Test {i+1}:")
    print(f"  Input: glucose_pre={glucose_pre} mg/dL, insulin={insulin} units, WW={ww}, fats={fats} g, prot={prot} g")
    print(f"  Time: hour={hour:.1f}, part_of_day={part_of_day_desc[part_of_day]}")
    print(f"  Dose assessment: {feedback_desc[feedback]} (insulin/WW: {test_case[5]:.2f})")
    print(f"  Suggested dose: {suggested_insulin:.1f} units")
    print(f"  Predicted glucose: {glucose_pred:.1f} mg/dL")
    print()


Test 1:
  Input: glucose_pre=150 mg/dL, insulin=2 units, WW=5, fats=10 g, prot=5 g
  Time: hour=9.5, part_of_day=Afternoon (12:00-17:59)
  Dose assessment: Slightly too low dose (insulin/WW: 0.40)
  Suggested dose: 6.1 units
  Predicted glucose: 148.2 mg/dL

Test 2:
  Input: glucose_pre=100 mg/dL, insulin=5 units, WW=3, fats=5 g, prot=2 g
  Time: hour=12.0, part_of_day=Morning (6:00-11:59)
  Dose assessment: OK (insulin/WW: 1.67)
  Suggested dose: 4.3 units
  Predicted glucose: 145.5 mg/dL

Test 3:
  Input: glucose_pre=200 mg/dL, insulin=15 units, WW=6, fats=20 g, prot=10 g
  Time: hour=18.5, part_of_day=Evening (18:00-23:59)
  Dose assessment: Too high dose (insulin/WW: 2.50)
  Suggested dose: 8.0 units
  Predicted glucose: 160.1 mg/dL

Test 4:
  Input: glucose_pre=90 mg/dL, insulin=2 units, WW=2, fats=5 g, prot=2 g
  Time: hour=22.0, part_of_day=Night (0:00-5:59)
  Dose assessment: Too low dose (insulin/WW: 1.00)
  Suggested dose: 3.8 units
  Predicted glucose: 183.7 mg/dL

Test 5:
 